In [45]:
import datetime as dt
import numpy as np
import os
import pandas as pd
import string

from common.models.reporter_model import ReporterModel
from common.models.read_file_model import ReadExlFileModel
from common.models.calc_model import Calc

headless_mode = True
url = 'http://sccts7dxsql/ctreport'
id = 'eisuke_koresawa'

# メンバリストファイル
member_list_path = r"\\mjs.co.jp\datas\CSC共有フォルダ\第47期 東京CSC第二グループ\47期SV共有\ダッシュボードアイテム"
member_list_file = 'メンバーリスト.xlsx'
member_list_file_path = os.path.join(member_list_path, member_list_file)

# 'クローズ_本日.xlsm'ファイル
close_file = r'\\mjs.co.jp\datas\CSC共有フォルダ\第47期 東京CSC第二グループ\47期SV共有\ダッシュボードアイテム\todays_op.xlsm'

# 活動ファイル
activity_path = r"\\mjs.co.jp\datas\CSC共有フォルダ\第47期 東京CSC第二グループ\47期SV共有\ダッシュボードアイテム"
activity_file = r"todays_activity.xlsx"
activity_file_path = os.path.join(activity_path, activity_file)

In [46]:
# レポータ、ファイル読込み、計算インスタンスを作成
rm = ReporterModel(url, id, headless_mode=headless_mode)
rf = ReadExlFileModel()
cl = Calc()

In [47]:
 # レポータから本日の各種情報を読込みDataFrameへ格納
df = rm.get_today_op_as_dataframe()

# メンバーリストファイルから、レポータの名前を正式な氏名のフォーマットに変換
member_list = pd.read_excel(member_list_file_path)
index_dict_reporter = member_list.set_index('レポータ')['氏名'].to_dict()
df.index = df.index.map(index_dict_reporter)

In [48]:
# クローズ_本日.xlsmファイルを読み込み、クローズのDataFrameを作成
df_close = rf.read_today_close_file()

# シフトデータを読込み、同時にメンバーリストファイルから、名前を正式な氏名のフォーマットに変換
df_shift = rf.read_today_shift_file()
index_dict_shift_table = member_list.set_index('Sweet')['氏名'].to_dict()
df_shift.index = df_shift.index.map(index_dict_shift_table)

In [49]:
# 当番データを読込み、同時にメンバーリストファイルから、名前を正式な氏名のフォーマットに変換
df_duty = rf.read_today_duty_file()
index_dict_duty_table = member_list.set_index('Sweet')['氏名'].to_dict()
df_duty.index = df_duty.index.map(index_dict_duty_table)
df_duty.to_excel('duty.xlsx')

In [50]:
# クローズデータとレポータからスクレイピングしたデータをjoin
df_join = df.join(df_close, how='outer').fillna(0)

In [51]:
# 各カラムの合計値を計算して新しい行として追加
df_join.loc['合計'] = df_join.sum()

In [52]:
# ACW, ATT, CPHを計算してカラムを追加
dataset = cl.calc_acw_att_cph(df_join)

In [53]:
# シフトデータをjoin
dataset = dataset.join(df_shift, how='outer').fillna('未設定')
dataset = dataset.join(df_duty, how='outer').fillna('')
dataset

,ログオン時間,着信通話時間の合計,着信通話時間の合計(外線),発信通話時間の合計,発信通話時間の合計(外線),ワークタイムの合計,着信後処理時間の合計,発信後処理時間の合計,離席時間の合計,待機時間,...,転送可時間の合計,他者支援時間の合計,開発資料確認時間の合計,資料作成時間の合計,ｸﾛｰｽﾞ,ACW,ATT,CPH,ｼﾌﾄ,業務ｽﾃｰﾀｽ
三ケ田 梨乃,0.0,0.010428,0.010428,0.001458,0.001458,0.000012,0.008588,0.000012,0.0,0.001412,...,0.0,0.000000,0.0,0.0,1.0,0.008611,0.011887,2.032750,209,
伊藤 克泰,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,週休,
内御堂 吉彦,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,週休,
北村 聡洋,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,週休,
原口 順子,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,週休,
合計,0.0,0.033762,0.033762,0.004444,0.004444,0.003623,0.026354,0.000104,0.0,0.543947,...,0.0,0.011192,0.0,0.0,10.0,0.004376,0.003821,5.083310,未設定,
吉岡 崇志,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,週休,
吉本 優成,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,週休,
吉田 祐二,0.0,0.002697,0.002697,0.001516,0.001516,0.002951,0.005856,0.000000,0.0,0.204005,...,0.0,0.000000,0.0,0.0,2.0,0.006892,0.002106,4.630225,322,
大竹 航平,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,週休,


In [54]:
# 必要なカラムのみ抽出
dataset = dataset[['ｼﾌﾄ', 'ACW', 'ATT', 'CPH', 'ｸﾛｰｽﾞ', '業務ｽﾃｰﾀｽ']]

In [55]:
# float型を'hh:mm:ss'形式の文字列に変換, CPHを小数点以下2桁でround
dataset = cl.apply_to_acw_att_cph(dataset)

C:\Users\eisuke_koresawa\Desktop\project\dashboard2\dashboard_app\common\models\calc_model.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['ACW'] = dataframe['ACW'].apply(self._float_to_hms)
C:\Users\eisuke_koresawa\Desktop\project\dashboard2\dashboard_app\common\models\calc_model.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['ATT'] = dataframe['ATT'].apply(self._float_to_hms)
C:\Users\eisuke_koresawa\Desktop\project\dashboard2\dashboard_app\common\models\calc_model.py:99: Se